In [1]:
import tarfile
import gzip
import scipy.io
import pandas as pd
import anndata as ad
import numpy as np
import scanpy as sc

In [2]:
def harmonise_genespace(adata, genes, keep_layer=[]):
    genes_to_add = []
    for g in genes:
        if g not in adata.var_names:
            genes_to_add += [g]
    print(f"{(len(genes)-len(genes_to_add))/len(genes)*100:.0f} % gene overlap with reference")
    adata = ad.AnnData(
        X=scipy.sparse.csr_matrix((adata.X.data, adata.X.indices, adata.X.indptr), shape=(adata.shape[0], adata.shape[1] + len(genes_to_add))),
        obs=adata.obs,
        var=pd.DataFrame(index=adata.var.index.tolist() + genes_to_add),
        layers={i:scipy.sparse.csr_matrix((adata.layers[i].data, adata.layers[i].indices, adata.layers[i].indptr), shape=(adata.shape[0], adata.shape[1] + len(genes_to_add))) for i in keep_layer}
    )[:, genes].copy()
    
    return adata

From: https://doi.org/10.1038/s41587-024-02157-8

Data: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE185472

In [3]:
basepath = "/storage/data/2404_revision/ce_data/gage_data"

In [4]:
adatas = []
with tarfile.open(f"{basepath}/GSE185472_RAW.tar", "r") as tar:
    sample_fns = [
        "GSM6679384_Hues6_5m_T", 
        "GSM6679385_822_5m_T", 
        "GSM6679386_HUES6_8m_GOGNT", 
        "GSM6679387_822_8m_GOGNT", 
        "GSM6679388_Hues6_10w_GOGN", 
        "GSM6679389_822_10w_GOGN", 
        "GSM6679390_Hues6_5m_GOGN", 
        "GSM6679391_822_5m_GOGN"
    ]
    for sample_fn in sample_fns:
        line, age, condition = sample_fn.split("_")[1:]
        obs = pd.read_csv(tar.extractfile(f"{sample_fn}_barcodes.tsv.gz"),
                          compression="gzip", header=None, index_col=0, names=[None])
        var = pd.read_csv(tar.extractfile(f"{sample_fn}_features.tsv.gz"),
                          compression="gzip", sep='\t', header=None, usecols=[0, 1], index_col=1, names=["ensembl_id", None])
        with gzip.open(tar.extractfile(f"{sample_fn}_matrix.mtx.gz"), 'rb') as mm:
            x = scipy.io.mmread(mm).T.tocsr().astype(np.float32)
        adata = ad.AnnData(X=x, obs=obs, var=var)
        adata.obs["sample"] = sample_fn
        adata.obs["line"] = line.capitalize()
        adata.obs["age"] = age
        adata.obs["condition"] = {"T": "transplant", "GOGNT": "transplant", "GOGN": "organoid"}[condition]
        adata.obs["suspension_type"] = "nucleus"
        if adata.shape[1] > 40000:
            adata = adata[:, adata.var.index.str.startswith("hg19_")].copy()
            adata.var.index = adata.var.index.str[5:]
            adata.var["ensembl_id"] = adata.var["ensembl_id"].str[5:]
        adata.var_names_make_unique()  
        adatas += [adata]

    sample_fns = [
        "GSM5615952_GOGNinvivo_822_6m_rep1.tar.gz",
        "GSM5615953_GOGNinvivo_HUES6_6m.tar.gz",
    ]
    for sample_fn in sample_fns:
        with tarfile.open(fileobj=tar.extractfile(sample_fn)) as tar2:
            obs = pd.read_csv(tar2.extractfile(f"{sample_fn[11:-7]}/outs/filtered_feature_bc_matrix/barcodes.tsv.gz"),
                              compression="gzip", header=None, index_col=0, names=[None])
            var = pd.read_csv(tar2.extractfile(f"{sample_fn[11:-7]}/outs/filtered_feature_bc_matrix/features.tsv.gz"),
                              compression="gzip", sep='\t', header=None, usecols=[0, 1], index_col=1, names=["ensembl_id", None])
            with gzip.open(tar2.extractfile(f"{sample_fn[11:-7]}/outs/filtered_feature_bc_matrix/matrix.mtx.gz"), "rb") as mm:
                x = scipy.io.mmread(mm).T.tocsr().astype(np.float32)
            adata = ad.AnnData(X=x, obs=obs, var=var)
            adata.obs["sample"] = sample_fn[:-7]
            adata.obs["line"] = sample_fn.split("_")[2].capitalize()
            adata.obs["age"] = "6m"
            adata.obs["condition"] = "transplant"
            adata.obs["suspension_type"] = "nucleus"
            adata = adata[:, adata.var.index.str.startswith("hg19_")].copy()
            adata.var.index = adata.var.index.str[5:]
            adata.var["ensembl_id"] = adata.var["ensembl_id"].str[5:]
            adata.var_names_make_unique()  
            adatas += [adata]

    sample_fn = "GSM7286711_Hues6_8mT_CTRL"
    obs = pd.read_csv(tar.extractfile("GSM7286711_barcodes_Hues6_8mT_CTRL.tsv.gz"),
                      compression="gzip", header=None, index_col=0, names=[None])
    var = pd.read_csv(tar.extractfile("GSM7286711_features_Hues6_8mT_CTRL.tsv.gz"),
                      compression="gzip", sep='\t', header=None, usecols=[0, 1], index_col=1, names=["ensembl_id", None])
    with gzip.open(tar.extractfile("GSM7286711_matrix_Hues6_8mT_CTRL.mtx.gz"), 'rb') as mm:
        x = scipy.io.mmread(mm).T.tocsr().astype(np.float32)
    adata = ad.AnnData(X=x, obs=obs, var=var)
    adata.obs["sample"] = sample_fn
    adata.obs["line"] = "Hues6"
    adata.obs["age"] = "8m"
    adata.obs["condition"] = "transplant"
    adata.obs["suspension_type"] = "cell"
    adata = adata[:, adata.var.index.str.startswith("hg19_")].copy()
    adata.var.index = adata.var.index.str[5:]
    adata.var["ensembl_id"] = adata.var["ensembl_id"].str[5:]
    adata.var_names_make_unique()
    adatas += [adata]
    
    
adata = ad.concat(adatas, index_unique="_")
adata.var_names_make_unique()

/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")
/home/ubuntu/venv/organoids/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names 

In [5]:
adata

AnnData object with n_obs × n_vars = 99127 × 32738
    obs: 'sample', 'line', 'age', 'condition', 'suspension_type'

In [6]:
hnoca_genes = pd.read_csv("/storage/data/2404_revision/hnoca_features.csv", index_col=0)
adata = harmonise_genespace(adata, hnoca_genes.index.tolist())
adata.var = hnoca_genes

53 % gene overlap with reference


In [7]:
obs_new = pd.DataFrame(index=adata.obs.index)

obs_new["sample_source"] = "3d_culture"
obs_new["organism"] = "Homo sapiens"
obs_new["disease"] = "healthy"

obs_new["cell_line_original"] = adata.obs["line"]
obs_new["organoid_age_days"] = adata.obs["age"].replace({"10w":70, "5m": 150, "6m": 180, "8m": 240})
obs_new["batch"] = adata.obs["sample"]
obs_new["treatment"] = adata.obs["condition"]
obs_new["cell_type_original"] = "unknown"

obs_new["suspension_type"] = adata.obs["suspension_type"]
obs_new["organ"] = "cerebral cortex"
obs_new["assay_sc"] = "10x 3' v3"
obs_new["ethnicity"] = "unknown"
obs_new["sex"] = obs_new["cell_line_original"].replace({"Hues6": "female", "822": "unknown"})
obs_new["development_stage"] = obs_new["cell_line_original"].replace({"Hues6": "blastula stage", "822": "unknown"})
obs_new["cell_type"] = "unknown"

obs_new["obs_names_original"] = obs_new.index
obs_new["publication"] = "Wang, 2024"
obs_new["doi"] = "10.1038/s41587-024-02157-8"
obs_new["hnoca_core"] = False

In [8]:
adata.obs = obs_new

adata.layers["counts_lengthnorm"] = adata.X.copy()
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)

In [9]:
ncells_before = adata.n_obs
adata = adata[(adata.X>0).sum(axis=1).A.ravel()>200].copy()
print(f"{(ncells_before-adata.n_obs)/ncells_before*100:.1f} % cells removed")

5.0 % cells removed


In [10]:
adata

AnnData object with n_obs × n_vars = 94129 × 36842
    obs: 'sample_source', 'organism', 'disease', 'cell_line_original', 'organoid_age_days', 'batch', 'treatment', 'cell_type_original', 'suspension_type', 'organ', 'assay_sc', 'ethnicity', 'sex', 'development_stage', 'cell_type', 'obs_names_original', 'publication', 'doi', 'hnoca_core'
    var: 'ensembl', 'gene_symbol'
    uns: 'log1p'
    layers: 'counts_lengthnorm'

In [11]:
adata.write(f"{basepath}/gage.h5ad", compression="gzip")